# 168 - Number Rotations

## Problem Statement

Consider the number $142857$. We can right-rotate this number by moving the last digit ($7$) to the front of it, giving us $714285$.<br>
It can be verified that $714285 = 5 \times 142857$.<br>

This demonstrates an unusual property of $142857$: it is a divisor of its right-rotation.

Find the last $5$ digits of the sum of all integers $n$, $10 \lt n \lt 10^{100}$, that have this property.

## Solution

First we can check the numbers with a low number of digits to see if a pattern emerges. We observe that there are two kinds of numbers. Some are numbers composed of a unique digit and the others. We can treat them separately.

For the numbers with a unique digit, the number, $S_k(d)$ with $k$ times the digit $d$ is given by 

\begin{equation}
    S_k(d) = d \left( \frac{10^k - 1}{9} \right).
\end{equation}

Summing over all the possible $d$ values, we get

\begin{equation}
    S_k = \sum_{d=1}^9 S_k(d).
\end{equation}

Summing over all possible number lengths between 10 and $10^9$, we get

\begin{equation}
    S = \sum_{k = 2}^{100} S_k.
\end{equation}

Then we observe the other numbers (those with several distinct digits). Checking for a sequence containing the first numbers, we find that they correspond to the [OEIS A034089](https://oeis.org/A034089) sequence. Let $p(q)$ denote the period of the fraction $q$ then the sequence is generated by $p(i / (10k-1))$, $k=2,3,4,5,6,7,8,9; k <= i <= 9$ and the concatenations of those periods.

Based on this, we simply need to generate the 36 values that are not concatenations using the formula above. We define a function to extract the period of the fractions. Then, we generate all the concatenations until there more than 100 digits. Along the way, we sum the last five digits.

Finally, the answer is the sum of the two parts.

In [1]:
from fractions import Fraction

for x in range(10, 1000000):
    s = str(x)
    rotated = s[-1] + s[:-1]
    if int(rotated) % x == 0:
        print(x)

11
22
33
44
55
66
77
88
99
111
222
333
444
555
666
777
888
999
1111
2222
3333
4444
5555
6666
7777
8888
9999
11111
22222
33333
44444
55555
66666
77777
88888
99999
102564
111111
128205
142857
153846
179487
205128
222222
230769
333333
444444
555555
666666
777777
888888
999999


In [2]:
for x in range(10, 1000000):
    s = str(x)
    rotated = s[-1] + s[:-1]
    if s != rotated and int(rotated) % x == 0:
        print(x)

102564
128205
142857
153846
179487
205128
230769


In [4]:
def find_periodic_part(numerator, denominator):
    """
    Compute the repeating decimal part of numerator / denominator.
    """
    remainder_seen = {}
    remainder = numerator % denominator
    position = 0
    periodic_part = ""

    while remainder != 0 and remainder not in remainder_seen:
        remainder_seen[remainder] = position
        numerator = remainder * 10
        digit = numerator // denominator
        periodic_part += str(digit)
        remainder = numerator % denominator
        position += 1

    if remainder == 0:
        return ""  # Terminating decimal
    else:
        start_pos = remainder_seen[remainder]
        return periodic_part[start_pos:]


def generate_values():
    results = []
    for k in range(2, 10):
        for i in range(k, 10):
            denominator = 10 * k - 1
            fraction = Fraction(i, denominator)
            period = find_periodic_part(fraction.numerator, fraction.denominator)
            results.append(int(period))
    return results


# Compute the sum of all numbers with distinct digits
unique_values = generate_values()
distinct_digits_sum = 0
for num in unique_values:
    s = str(num)
    n = len(s)
    i = 1
    while n * i < 101:
        distinct_digits_sum += num % 10**5
        distinct_digits_sum %= 10**5
        i += 1

# Generate the sum of all numbers with unique digit
unique_digits_sum = 0
for k in range(2, 101):
    sum_k = 0
    for d in range(1, 10):
        sum_k += d * (10**k - 1) // 9
    unique_digits_sum += sum_k % 10**5
    unique_digits_sum %= 10**5

# Sum the two parts and take modulo
(unique_digits_sum + distinct_digits_sum) % 10**5

59206